In [2]:
import pandas as pd
import numpy as np
from scipy import stats
import os
import glob
import re
import io
import matplotlib.pyplot as plt
from matplotlib import rc
import statsmodels.api as sm

In [3]:
df = pd.read_excel('T5-R23-D103-L25_photocurrent.xlsx')


In [4]:
df

,Unnamed: 0,Cross,Bar,Coupler length (um),Input Power (mw),Input Power Error (mw),Photocurrent (mA),Photocurrent Error (mA),Output Power (mW),Output Power Error (mW)
0,0,0,1,25,29.01,0.05,0.798281,1.704486e-03,0.688045,2.634783e-06
1,1,1,1,25,29.01,0.05,0.059149,2.778712e-04,0.059556,2.028089e-07
2,2,2,1,25,29.01,0.05,0.003987,1.386413e-05,0.003900,7.394022e-09
3,3,3,1,25,29.01,0.05,0.000236,9.544628e-07,0.000270,1.272274e-09
4,5,4,1,25,29.01,0.05,0.000016,7.550507e-08,0.000023,2.181578e-10


In [5]:
df['log_fractional_power'] = np.log10(df['Output Power (mW)']/df['Input Power (mw)'])
df['log_photocurrent'] = np.log10(df['Photocurrent (mA)'])

In [8]:
# Propogation of measurement error through a log function
fractional_error_output_power = df['Output Power Error (mW)']/df['Output Power (mW)']
fractional_error_input_power = df['Input Power Error (mw)']/df['Input Power (mw)']
log_fractional_power_error = np.sqrt(fractional_error_input_power**2 + fractional_error_output_power**2)/(np.log(10))
df['log_fractional_power_error'] = log_fractional_power_error

log_photocurrent_error = (1/np.log(10))*(df['Photocurrent Error (mA)']/df['Photocurrent (mA)'])
df['log_photocurrent_error'] = log_photocurrent_error


In [9]:
df

,Unnamed: 0,Cross,Bar,Coupler length (um),Input Power (mw),Input Power Error (mw),Photocurrent (mA),Photocurrent Error (mA),Output Power (mW),Output Power Error (mW),log_fractional_power,log_photocurrent,log_fractional_power_error,log_photocurrent_error
0,0,0,1,25,29.01,0.05,0.798281,1.704486e-03,0.688045,2.634783e-06,-1.624931,-0.097844,0.000749,0.000927
1,1,1,1,25,29.01,0.05,0.059149,2.778712e-04,0.059556,2.028089e-07,-2.687622,-1.228049,0.000749,0.002040
2,2,2,1,25,29.01,0.05,0.003987,1.386413e-05,0.003900,7.394022e-09,-3.871490,-2.399364,0.000749,0.001510
3,3,3,1,25,29.01,0.05,0.000236,9.544628e-07,0.000270,1.272274e-09,-5.030898,-3.626674,0.000749,0.001755
4,5,4,1,25,29.01,0.05,0.000016,7.550507e-08,0.000023,2.181578e-10,-6.106379,-4.801085,0.000749,0.002074


In [10]:
Y = df['log_photocurrent']
X = df['Cross']
wt = 1/(df['log_photocurrent_error'])**2

x = sm.add_constant(X)

fit_wls = sm.WLS(Y,x, weights=wt).fit()

In [11]:
print(fit.summary())

                            OLS Regression Results                            
Dep. Variable:       log_photocurrent   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 1.613e+04
Date:                Wed, 14 Jun 2023   Prob (F-statistic):           1.08e-06
Time:                        13:47:02   Log-Likelihood:                 11.817
No. Observations:                   5   AIC:                            -19.63
Df Residuals:                       3   BIC:                            -20.42
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0696      0.023     -3.056      0.0

c:\Users\karthiksp\Anaconda3\envs\data\lib\site-packages\statsmodels\stats\stattools.py:74: ValueWarning: omni_normtest is not valid with less than 8 observations; 5 samples were given.
  warn("omni_normtest is not valid with less than 8 observations; %i "


In [15]:

print(fit_wls.summary())

                            WLS Regression Results                            
Dep. Variable:       log_photocurrent   R-squared:                       1.000
Model:                            WLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 1.768e+04
Date:                Wed, 14 Jun 2023   Prob (F-statistic):           9.38e-07
Time:                        13:58:03   Log-Likelihood:                 11.555
No. Observations:                   5   AIC:                            -19.11
Df Residuals:                       3   BIC:                            -19.89
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0876      0.017     -5.233      0.0

c:\Users\karthiksp\Anaconda3\envs\data\lib\site-packages\statsmodels\stats\stattools.py:74: ValueWarning: omni_normtest is not valid with less than 8 observations; 5 samples were given.
  warn("omni_normtest is not valid with less than 8 observations; %i "


In [17]:
contst, slope, = fit_wls.params
const_err, slope_err = fit_wls.bse

In [18]:
slope

-1.1731948987224938